<a href="https://colab.research.google.com/github/alzimna/Portfolio/blob/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Overview**

Pada akhir tahun 2023, hubungan antara Palestina dan Israel memanas akibat penyerangan Israel di Gaza, Palestina. Penyerangan ini tidak hanya berdampak pada rakyat Palestina saja namun juga memberikan bekas luka kepada rakyat muslim di seluruh dunia. Hal ini memicu perlawanan dari seluruh negara muslim termasuk Indonesia sendiri.

Di Indonesia, santer terdengar pernyataan boikot terhadap produk yang diduga menyalurkan dana kepada Israel. Setelah kami lakukan investigasi di Twitter, perusahaan seperti Unilever dan KHC menjadi topik perbincangan teratas.

Projek ini merupakan projek yang dilakukan oleh saya dan 3 orang teman saya. Tujuan dari produk ini adalah menginvestigasi efek dari boikot melalui analisis sentimen di Twitter pada kenaikan harga sahan dari Unilever (UNVR) lau membandingkannya dengan perusahaan yang terkena boikot namun tidak menjadi trending topic di twitter seperti KHC.

# **Project Method**
Berikut ini langkah-langkah yang dilakukan pada projek kali ini.

1. Scrapping tweet dengan kata kunci "UNVR", "Unilever", "boikot unilever".
2. Data preprocessing seperti
  * Menghapus duplikat
  * Mengubah format tanggal
  * Menghilangkan simbol khusus seperti \r, \n, ", url, dan byte
  * Mengubah menjadi lower case
  * Menghapus simbol dan tanda baca
  * Menormalisasi kata
  * Menghapus kata tidak bermakna
  * Menghapus imbuhan
  * Tokenize data
3. Analisis data eksploratif
4. Mengambil data saham UNVR dan KHC dari yahoo finance
5. Melakukan analisis sentimen pada tiap tweet
6. Mengklasifikasikan hari berdasarkan kejadian Naik/Turunnya harga saham UNVR
7. Melakukan klasifikasi pada data yang dilengkapi polarity dan tidak untuk dibandingkan akurasinya. Model yang dipakai antara lain Regresi Logistik, Linear Discriminant Analysis (LDA), KNN, dan SVM.

Kesimpulan yang dapat diperoleh dari projek ini adalah **Polarity akan menaikkan akurasi model jika dibandingkan dengan data tanpa polaritas. Artinya analisis sentimen dapat membantu untuk menaikkan akurasi model biasa untuk mengklasifikasian Naik/Turunnya harga saham**




# **Code Snippet**

In [52]:
# Import Libraries

# For EDA
import pandas as pd
import numpy as np
import re
import missingno as msno

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import nltk
import time
from tqdm import tqdm

#For Pre-Processing
import torch
import torchvision
import random
import os

import plotly.express as px

import nltk
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
nltk.download("punkt_tab")
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
!pip install contractions
import contractions
import re
import string

from textblob import TextBlob
from IPython.display import clear_output
!pip install googletrans
from googletrans import Translator
translator = Translator()

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Data berikut ini merupakan hasil scrapping tweet dengan kata kunci "UNVR", "Unilever", dan "boikot unilever". Ketiga data tersebut disatukan untuk dilakukan preprocessing.

In [53]:
url = {}
url[1] = "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/Data_UNVR.xlsx"
url[2] = 'https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/Unilever.xlsx'
url[3] = 'https://github.com/alzimna/Portfolio/blob/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/boikot_unilever.xlsx'

df = pd.DataFrame()
for i in range(1,3) :
  temp = pd.read_excel(url[i])
  df = pd.concat([df,temp],axis = 0)

data = df[["created_at","username","tweet_url","full_text"]]
data = data.rename(columns = {
    "created_at" : "Tanggal",
    "tweet_url" : "Link",
    "full_text" : "Konten"
})

data.reset_index(inplace = True,drop = True)
data.head(20)

,Tanggal,username,Link,Konten
0,Wed Dec 06 22:34:21 +0000 2023,ataritafaran,https://twitter.com/ataritafaran/status/173252...,@rasyidrobbani @anang_kur @rykarlsen Kalau ord...
1,Wed Dec 06 22:04:00 +0000 2023,nooptionmood,https://twitter.com/nooptionmood/status/173252...,@rykarlsen Sepupu Di unvr infonya masih stabil...
2,Wed Dec 06 16:07:27 +0000 2023,rnamidubx,https://twitter.com/rnamidubx/status/173243157...,@rykarlsen Pt kantor sy ga kena boikot dan ga ...
3,Wed Dec 06 00:35:52 +0000 2023,doktermarket,https://twitter.com/doktermarket/status/173219...,Pagi. semalam bursa AS ditutup bervariasi. Dow...
4,Tue Dec 05 23:23:47 +0000 2023,a37723_fauzan,https://twitter.com/a37723_fauzan/status/17321...,melihat adanya kenaikan yang stabil dalam bebe...
5,Tue Dec 05 05:41:08 +0000 2023,Bisniscom,https://twitter.com/Bisniscom/status/173191156...,"4 Direksi Unilever Indonesia (UNVR) Mundur, Be..."
6,Tue Dec 05 05:37:10 +0000 2023,KATADATAcoid,https://twitter.com/KATADATAcoid/status/173191...,"IHSG Sesi I Turun 0,4%, Saham UNVR hingga GOTO..."
7,Mon Dec 04 23:50:11 +0000 2023,InvestorID,https://twitter.com/InvestorID/status/17318232...,MNC Sekuritas memprediksi IHSG hari ini terkor...
8,Mon Dec 04 23:41:04 +0000 2023,Bisniscom,https://twitter.com/Bisniscom/status/173182094...,"IHSG Hari Ini Menuju 7.130, Cermati Saham BBCA..."
9,Mon Dec 04 09:06:49 +0000 2023,Iilasikuta,https://twitter.com/Iilasikuta/status/17316009...,@parxhanbin @Nakyunga IYAAA!! KENAPA UNVR


# **Data Preprocessing**

# Menghapus duplikat

In [54]:
# Pengecekan data duplikasi
print("data duplikat ada :",data.duplicated().sum())

data = data.drop_duplicates()

# Pengecekan data duplikasi kedya
print("data duplikat ada :", data.duplicated().sum())

#Menghapus baris yang memuat na
data = data.dropna()

data.info()

data duplikat ada : 79
data duplikat ada : 0
<class 'pandas.core.frame.DataFrame'>
Index: 6213 entries, 0 to 6293
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Tanggal   6213 non-null   object
 1   username  6213 non-null   object
 2   Link      6213 non-null   object
 3   Konten    6213 non-null   object
dtypes: object(4)
memory usage: 242.7+ KB


# Mengubah format tanggal

In [55]:
from datetime import datetime
temp = []
for i in range(data.shape[0]) :
  cek = data.iloc[i]["Tanggal"]
  s = cek[8:10]+" "+cek[4:7]+" "+cek[-4:]
  s = datetime.strptime(s, "%d %b %Y")
  temp.append(s)
data["Tanggal"] = temp

data = data.sort_values(by='Tanggal')
data.reset_index(inplace = True,drop = True)
data.head()

,Tanggal,username,Link,Konten
0,2023-10-07,ijcancer_,https://twitter.com/ijcancer_/status/171048510...,"Mantan presbem, kerja di unilever, keterima di..."
1,2023-10-07,MakassarFess,https://twitter.com/MakassarFess/status/171048...,Tabe info info yg bingung mau kemana weekendny...
2,2023-10-07,PurazaAnalytics,https://twitter.com/PurazaAnalytics/status/171...,Sebesar besar company macam Unilever pun guna ...
3,2023-10-07,selembutbolu,https://twitter.com/selembutbolu/status/171070...,@pengwinningg Mangat unilever ðŸ¤­ðŸ’‹ðŸ’‹ðŸ’‹
4,2023-10-07,ncaaidnzz,https://twitter.com/ncaaidnzz/status/171069358...,@UPIfess unilever AAMIINN PLEASEEEE selain dek...


# Menghilangkan simbol khusus seperti \r, \n, ", url, dan byte

In [56]:
# TEXT PRE-PROCESSING
# TAHAP 1
# Menghilangkan simbol khusus seperti \r, \n, ", url, dan byte
def cleaning(data):
  data['Konten_Parsed_1'] = data['Konten'].str.replace("\\\\r",
  " ")
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace("\\\\n", " ")
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace("\n", " ")
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace("\r", " ")
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace("&amp", " ")
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace("&gt", " ")
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace("&lt", " ")
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace(" ", " ")
  data['Konten_Parsed_1']= data['Konten_Parsed_1'].str.replace(r'''(?i)\b((?:https|http?:/
  /|www\d{0,3}[.]|[a-z0-9.\-]+[.][az]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:
  \(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”
  ‘’]))''', " ")
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace('"', '')
  data['Konten_Parsed_1'] = data['Konten_Parsed_1'].str.replace("\\\\x[a-zA-z0-9][a-zA-z0-9]","")
  return data

data = cleaning(data)
print("Sampel data sebelum preprocess :\n",data["Konten"][0])
print("Hasil setelah tahap 1 :\n",data["Konten_Parsed_1"][0])

Sampel data sebelum preprocess :
 Mantan presbem, kerja di unilever, keterima di pbb gadiambil karena pengennya di unicef,  mau nyoba ke Freeport. Disukai temen kantor, cewek UI, pinter, volunteer luar negeri, cantik lagi. Sempurna banget gak tu idup, lupa nanya linkedinnya ðŸ¥², jaringan keputus
Hasil setelah tahap 1 :
 Mantan presbem, kerja di unilever, keterima di pbb gadiambil karena pengennya di unicef,  mau nyoba ke Freeport. Disukai temen kantor, cewek UI, pinter, volunteer luar negeri, cantik lagi. Sempurna banget gak tu idup, lupa nanya linkedinnya ðŸ¥², jaringan keputus


# Mengubah menjadi lower case

In [57]:
#TAHAP 2
# Mengubah menjadi lower case
def lowerCase(data):
    data['Konten_Parsed_2'] = data['Konten_Parsed_1'].str.lower()
    return data
data = lowerCase(data)
print("Sampel data sebelum preprocess :\n",data["Konten"][0])
print("Hasil setelah tahap 2 :\n",data["Konten_Parsed_2"][0])

Sampel data sebelum preprocess :
 Mantan presbem, kerja di unilever, keterima di pbb gadiambil karena pengennya di unicef,  mau nyoba ke Freeport. Disukai temen kantor, cewek UI, pinter, volunteer luar negeri, cantik lagi. Sempurna banget gak tu idup, lupa nanya linkedinnya ðŸ¥², jaringan keputus
Hasil setelah tahap 2 :
 mantan presbem, kerja di unilever, keterima di pbb gadiambil karena pengennya di unicef,  mau nyoba ke freeport. disukai temen kantor, cewek ui, pinter, volunteer luar negeri, cantik lagi. sempurna banget gak tu idup, lupa nanya linkedinnya ðÿ¥², jaringan keputus


# Menghapus simbol dan tanda baca

In [58]:
#TAHAP 3
#Menghapus simbol dan tanda baca
def removePunct(data):
    data['Konten_Parsed_3'] = data['Konten_Parsed_2']
    data['Konten_Parsed_3'] = data['Konten_Parsed_3'].str.replace("'s","")
    data['Konten_Parsed_3'] = data['Konten_Parsed_3'].str.replace('[^a-zA-Z0-9]+',' ')
    return data
data = removePunct(data)
print("Sampel data sebelum preprocess :\n",data["Konten"][0])
print("Hasil setelah tahap 3 :\n",data["Konten_Parsed_3"][0])

Sampel data sebelum preprocess :
 Mantan presbem, kerja di unilever, keterima di pbb gadiambil karena pengennya di unicef,  mau nyoba ke Freeport. Disukai temen kantor, cewek UI, pinter, volunteer luar negeri, cantik lagi. Sempurna banget gak tu idup, lupa nanya linkedinnya ðŸ¥², jaringan keputus
Hasil setelah tahap 3 :
 mantan presbem, kerja di unilever, keterima di pbb gadiambil karena pengennya di unicef,  mau nyoba ke freeport. disukai temen kantor, cewek ui, pinter, volunteer luar negeri, cantik lagi. sempurna banget gak tu idup, lupa nanya linkedinnya ðÿ¥², jaringan keputus


# Menormalisasi kata

In [ ]:
#TAHAP 4
#Normalisasi setiap kata
url1 = 'https://github.com/nasalsabila/kamus-alay/raw/master/colloquial-indonesian-lexicon.csv'
kamus1 = pd.read_csv(url1).iloc[:,:2]
kamus1 = kamus1.rename(columns={"slang": "non_standar","formal" : "standar"})

#Normalisasi tahap 1
def normalize_1(text):
    nonstdword = kamus1['non_standar'].values.tolist()
    stdword = kamus1['standar'].values.tolist()
    text = text.split(" ")
    for i in range(len(text)):
      if text[i] in nonstdword:
        index = nonstdword.index(text[i])
        text[i] = stdword[index]
    return ' '.join(map(str, text))

data['Konten_Parsed_4'] = data['Konten_Parsed_3']
data['Konten_Parsed_4'] = data['Konten_Parsed_4'].map(lambda com : normalize_1(com))

# Menghilangkan angka
data['Konten_Parsed_4'] = data['Konten_Parsed_4'].str.replace('[^a-zA-Z]+',' ')

# Menghapus Multiple Whitespace
def remove_multiple_whitespace(text):
  text = re.sub('\s+',' ',text)
  return text.translate(str.maketrans("","",string.punctuation))

data['Konten_Parsed_4'] = data['Konten_Parsed_4'].apply(remove_multiple_whitespace)
print("Sampel data sebelum preprocess :\n",data["Konten"][2])
print("Hasil setelah tahap 4 :\n",data["Konten_Parsed_4"][2])

## Menghapus kata tidak bermakna

In [ ]:
# TAHAP 5
# Menghilangkan kata tidak bermakna (stopwords)
import base64
import requests
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopwords_sastrawi = factory.get_stop_words()

master = "https://github.com/masdevid/ID-Stopwords/raw/master/id.stopwords.02.01.2016.txt"
req = requests.get(master)
req = req.text

meaningless = req.split("\n")
stopwords_nltk = list(stopwords.words('indonesian')) + list(stopwords.words('english'))
more_stopword = ['gue',"beliau","doang","ha","guia","engkau","ku","sih","nya",
                 "kayak","tau","pikir","bikin","kali","gara","nama",
                 "bilang","tinggal","tanyakanrl","deh","iya","biar","pa"]
meaningless = meaningless + stopwords_nltk + stopwords_sastrawi+more_stopword

def delete_meaningless(text) :
  result = []
  text = text.split(" ")
  for i in range(len(text)):
    if text[i] not in meaningless:
      result.append(text[i])
  return ' '.join(map(str,result))
data['Konten_Parsed_5'] = data['Konten_Parsed_4'].map(lambda com : delete_meaningless(com))
print("Sampel data sebelum preprocess :\n",data["Konten"][2])
print("Hasil setelah tahap 5 :\n",data["Konten_Parsed_5"][2])

# Menghilangkan Imbuhan (stemming)
Karena data terlalu besar nantinya akan membuat proses stemming menjadi sangat lama, sehingga kami membaginya menjadi 4 bagian dan melakukan stemming secara terpisah. Setelah mendapatkan hasil stemming, data disatukan kembali seperti pada sintaks berikut ini.

In [ ]:
# TAHAP 6 Lemmatizating dan Stemming
# Menghilangkan imbuhan (stemming)
wn= nltk.WordNetLemmatizer()
def lemmatization(text):
  result = []
  text = text.split(" ")
  for word in text :
    result.append(wn.lemmatize(word))
  return ' '.join(map(str,result))

# Buat kolom tambahan untuk data description yang telah dilemmatization
data['Konten_Parsed_6'] = data['Konten_Parsed_5'].apply(lemmatization)

index_to_drop = []
for i in range(data.shape[0]) :
  text = data["Konten_Parsed_6"][i]
  text = text.split(" ")
  if len(text)==1 :
    index_to_drop.append(i)
data = data.drop(index_to_drop).reset_index(drop = True)

In [ ]:
# from IPython.display import clear_output

# url = 'https://github.com/alzimna/Pemodelan_UAS/raw/main/data_sebelum_stem.xlsx'
# a = 4500
# b = 6208
# bagianku = pd.read_excel(url)[a:b]
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()
# def stemming(text):
#   result = []
#   text = text.split(" ")
#   for word in text :
#     stem_text = stemmer.stem(word)
#     result.append(stem_text)
#   result = ' '.join(map(str,result))
#   return result

# stem = []
# n = bagianku.shape[0]
# for i in range(n) :
#   stem.append(stemming(bagianku.iloc[i]["Konten_Parsed_6"]))
#   print(i,"/",n)
#   clear_output(wait=True)

# # Buat kolom tambahan untuk data description yang telah dilemmatization
# bagianku['Konten_Parsed_6'] = stem

# bagianku.to_excel("bagianku_setelah_stem.xlsx",index = None)

In [ ]:
url = ["https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/bagian_rara.xlsx",
       "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/bagian_maya.xlsx",
       "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/bagian_raisa.xlsx",
       "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/bagian_alzim.xlsx"

]
data = pd.DataFrame()
for i in range(4) :
  temp = pd.read_excel(url[i])
  data = pd.concat([data,temp],axis = 0)
data.reset_index(inplace = True,drop = True)
print("Sampel data sebelum preprocess :\n",data.iloc[2]["Konten"])
print("Hasil setelah tahap 6 :\n",data.iloc[2]["Konten_Parsed_6"])

In [ ]:
#Menghapus stopword setelah distemming
factory = StopWordRemoverFactory()
stopwords_sastrawi = factory.get_stop_words()

master = "https://github.com/masdevid/ID-Stopwords/raw/master/id.stopwords.02.01.2016.txt"
req = requests.get(master)
req = req.text

meaningless = req.split("\n")
stopwords_nltk = list(stopwords.words('indonesian')) + list(stopwords.words('english'))
more_stopword = ['gue',"beliau","doang","ha","guia","engkau","ku","sih","nya",
                 "kayak","tau","pikir","bikin","kali","gara","nama",
                 "bilang","tinggal","tanyakanrl","deh","iya","biar","pa",
                 "cv","p","wa","jasa","jokitugas","cover","itv","hm","jasacvats","kak","tanyarlfes",
                 "surat","lamar","republikaonline","cek","letter","warpin","convomf"]
meaningless = meaningless + stopwords_nltk + stopwords_sastrawi+more_stopword

data['Konten_Parsed_6'] = data['Konten_Parsed_6'].apply(delete_meaningless)

# Tokenize Data

In [ ]:
# Tokenisasi
def tokenize(text):
  token = nltk.word_tokenize(text)
  return token
data["tokenize"] = data["Konten_Parsed_6"].apply(tokenize)

In [ ]:
# Fixing Data
data = data.drop(['Konten_Parsed_1','Konten_Parsed_2','Konten_Parsed_3','Konten_Parsed_4',
           'Konten_Parsed_5'],axis = 1)
data = data.rename(columns={'Konten_Parsed_6' : 'Konten_Parsed'})

print("Sampel data sebelum preprocess :\n",data.iloc[2]["Konten"])
print("Hasil setelah tahap 6 :\n",data.iloc[2]["Konten_Parsed"])

In [ ]:
data.head(20)

In [ ]:
# ANALISIS DATA EKSPLORATIF
data_eda = data.copy()
data_eda.info()
data_eda = pd.DataFrame(data_eda)

In [ ]:
# Membuat grafik untuk 10 kata teratas yang paling banyak muncul di seluruh korpus
# Gabungkan teks yang telah diproses sebelumnya dari semua isi berita
text = ' '.join(data_eda['Konten_Parsed'])

# Tokenisasi
words = text.split()

# Hitung frekuensi setiap kata
word_freq = pd.Series(words).value_counts()

# Pilih 10 kata teratas yang paling sering
top_words = word_freq.head(50)

#print
print(top_words)

In [ ]:
sns.set_theme(palette="muted")
fig = plt.figure(figsize = (10,6))
ax = sns.barplot(x = top_words.index[:10], y = top_words.values[:10])
# Adding labels for every bar
for bar, label in zip(ax.patches, top_words.values):
    ax.annotate(label, (bar.get_x() + bar.get_width() / 2, bar.get_height()), ha='center', va='bottom')

In [ ]:
# Perbandingan jumlah kata sebelum dan sesudah text preprocessing
text1 = ' '.join(data['Konten'])
print('Total kata sebelum pre-processing adalah sebanyak {} kata'.format(len(text1)))
text2 = ' '.join(data['Konten_Parsed'])
print('Total kata setelah pre-processing adalah sebanyak {} kata'.format(len(text2)))

In [ ]:
# WORD CLOUDS
text = ' '.join(data['Konten_Parsed'])
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig('foo.png')
plt.show()

# Loading data UNVR dan KHC

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

# Define the start and end dates
start_date = datetime(2023, 8, 7)
end_date = datetime(2023, 12, 7)

# Define the interval between datetime objects
interval = timedelta(days=1)

# Create a list of datetime objects within the specified range
tanggal = [start_date + i * interval for i in range((end_date - start_date).days + 1)]

In [ ]:
url = "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/Saham%20KHC.xlsx"
khc = pd.read_excel(url)
khc_full = {}
for hari in tanggal :
  if hari in list(khc["Date"]) :
    khc_full[hari] = float(khc[khc["Date"]==hari]["Adj Close"])
  else :
    khc_full[hari] = 0
for hari in khc_full.keys() :
  if khc_full[hari] == 0 :
    if khc_full[hari+interval] != 0:
      khc_full[hari] = 1/2*(khc_full[hari-interval]+khc_full[hari+interval])
    else :
      khc_full[hari] = 1/3*(khc_full[hari-interval]+2*khc_full[hari+2*interval])
khc = pd.DataFrame({
    "Date" : khc_full.keys(),
    "Adj" : khc_full.values()
})
khc

In [ ]:
Label = []
n = khc.shape[0]
for i in range(1,n) :
  if khc.iloc[i,1] >= khc.iloc[i-1,1] :
    Label.append("Naik")
  else :
    Label.append("Turun")
khc.drop(0,inplace = True)
khc["Label"] = Label

In [ ]:
bound = datetime(2023, 10, 7)
n = khc.shape[0]

n_b = 0
n_a = 0
t_b = 0
t_a = 0
for i in range(n) :
  if khc.iloc[i,0] < bound :
    if khc.iloc[i,2] == "Naik" :
      n_b += 1
    else :
      t_b +=1
  else :
    if khc.iloc[i,2] == "Naik" :
      n_a += 1
    else :
      t_a +=1
df = pd.DataFrame({
    "Naik" : [n_b,n_a],
    "Turun" : [t_b,t_a]
}, index = ["Sebelum boikot","Sesudah boikot"])
df

In [ ]:
url = "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/UNVR.JK.xlsx"
unvr = pd.read_excel(url)
unvr_full = {}
for hari in tanggal :
  if hari in list(unvr["Date"]) :
    unvr_full[hari] = float(unvr[unvr["Date"]==hari]["Adj Close"])
  else :
    unvr_full[hari] = 0
for hari in unvr_full.keys() :
  if unvr_full[hari] == 0 :
    if unvr_full[hari+interval] != 0:
      unvr_full[hari] = 1/2*(unvr_full[hari-interval]+unvr_full[hari+interval])
    else :
      unvr_full[hari] = 1/3*(unvr_full[hari-interval]+2*unvr_full[hari+2*interval])
unvr = pd.DataFrame({
    "Date" : unvr_full.keys(),
    "Adj" : unvr_full.values()
})
unvr

In [ ]:
Label = []
n = unvr.shape[0]
for i in range(1,n) :
  if unvr.iloc[i,1] >= unvr.iloc[i-1,1] :
    Label.append("Naik")
  else :
    Label.append("Turun")
unvr.drop(0,inplace = True)
unvr["Label"] = Label

In [ ]:
unvr

In [ ]:
bound = datetime(2023, 10, 7)
n = unvr.shape[0]

n_b = 0
n_a = 0
t_b = 0
t_a = 0
for i in range(n) :
  if unvr.iloc[i,0] < bound :
    if unvr.iloc[i,2] == "Naik" :
      n_b += 1
    else :
      t_b +=1
  else :
    if unvr.iloc[i,2] == "Naik" :
      n_a += 1
    else :
      t_a +=1
df = pd.DataFrame({
    "Naik" : [n_b,n_a],
    "Turun" : [t_b,t_a]
}, index = ["Sebelum boikot","Sesudah boikot"])
df

# Analisis Sentimen Setiap Tweet
Perhatikan data setelah setemming berikut ini

In [ ]:
data.head(5)

Setiap tweet mempunyai tiga jenis tendensi, yaitu positif (nilai polarity positif), netral (nilai polarity 0), dan negatif (nilai polarity negatif). Langkah selanjutnya adalah mencari nilai polarity dari setiap tweet menggunakan package TextBlob. Untuk menggunakan TextBlob, tweet berbahasa Indonesia diterjemahkan kedalam bahasa inggris terlebih dahulu. Karena data terlalu besar, kami membaginya menjadi 4 bagian. Berikut ini diberikan contoh untuk mendapatkan nilai polarity setiap tweet.

In [ ]:
bagianku = data[:100]
bagianku.reset_index(drop = True,inplace = True)
polarity = []
n = 100
cek = bagianku.iloc[1]["Konten_Parsed"]
translation = await translator.translate(cek,src = "id",dest='en')
text_indo = TextBlob(translation.text)
text_indo.sentiment
for i in range(n) :
  cek = bagianku.iloc[i]["Konten_Parsed"]
  translation = await translator.translate(cek,src = "id",dest='en')
  text_indo = TextBlob(translation.text)
  analysis = text_indo.sentiment
  polarity.append(analysis.polarity)
  print(i,"/",n)
  clear_output(wait=True)
bagianku["polarity"] = polarity
bagianku.head(10)

In [ ]:
url = [
    "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/bagian_rara_polarity.xlsx",
    "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/bagian_maya_polarity.xlsx",
    "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/bagian_raisa_polarity.xlsx",
    "https://github.com/alzimna/Portfolio/raw/refs/heads/main/Stock%20Price%20Prediction%20with%20Sentiment%20Analysis/data/bagian_alzim_polarity.xlsx"

]
data_polarity = pd.DataFrame()
for i in range(len(url)) :
  temp = pd.read_excel(url[i])
  data_polarity = pd.concat([data_polarity,temp],axis = 0)
data_polarity.reset_index(inplace = True,drop = True)
data_polarity

In [ ]:
def analyze_sentiment(polarity):
    if polarity < 0:
        return 'Negatif'
    elif polarity == 0:
        return 'Netral'
    else:
        return 'Positif'

# Add a new column 'Sentimen' to the DataFrame based on sentiment analysis
data_polarity['Sentimen'] = data_polarity['polarity'].apply(analyze_sentiment)

# Count the number of positive, negative, and neutral sentiments
sentiment_counts = data_polarity['Sentimen'].value_counts()

# Calculate the percentages
total_entries = len(data_polarity)
percentage_positif = (sentiment_counts.get('Positif', 0) / total_entries) * 100
percentage_negatif = (sentiment_counts.get('Negatif', 0) / total_entries) * 100
percentage_netral = (sentiment_counts.get('Netral', 0) / total_entries) * 100

print("Persentase sentimen:")
print(f"Positif: {percentage_positif:.2f}%")
print(f"Negatif: {percentage_negatif:.2f}%")
print(f"Netral: {percentage_netral:.2f}%")

Setelah mendapatkan nilai polarity setiap tweet, dihitung rata-rata polarity dari tweet pada hari yang sama. Kemudian disatukan dengan data harga saham dan label kenaikannya.

In [ ]:
p = data_polarity.groupby("Tanggal")["polarity"].mean().reset_index()
unvr = unvr[unvr["Date"] >= datetime(2023, 10, 7)].reset_index(drop = True)
unvr = pd.concat([unvr,p.iloc[:,1]],axis = 1)
unvr

In [ ]:
unvrp = unvr.drop(['Adj','Label'], axis = 1)
unvrp = pd.DataFrame(unvrp)
unvrp.head()

In [ ]:
# Membuat kolom label
unvr['Label'] = unvr['Adj'].shift(-1)  # Shift harga satu baris ke atas
unvr['Label'] = unvr.apply(lambda row: 'Naik' if row['Label'] > row['Adj'] else 'Turun', axis=1).shift(+1)
unvr = unvr.drop(0).reset_index(drop = True)
unvr

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Initialize StandardScaler
scaler = MinMaxScaler()
unvr["Adj"] = scaler.fit_transform(unvr[["Adj"]])
unvr.head()

In [ ]:
import matplotlib.pyplot as plt
unvr.plot(x='Date', y=['Adj', 'polarity'], kind='line', marker='o')
plt.show()

In [ ]:
class_tanpa_polarity = unvr.iloc[3:,].copy().drop(["Adj","Date","polarity"],axis = 1).reset_index(drop = True)
n = class_tanpa_polarity.shape[0]
adj_d1 = []
adj_d2 = []
adj_d3 = []
for i in range(n) :
  adj_d1.append(unvr.iloc[i+2,1])

for i in range(n) :
  adj_d2.append(unvr.iloc[i+1,1])

for i in range(n) :
  adj_d3.append(unvr.iloc[i,1])
class_tanpa_polarity["adj_d1"] = adj_d1
class_tanpa_polarity["adj_d2"] = adj_d2
class_tanpa_polarity["adj_d3"] = adj_d3
class_tanpa_polarity

In [ ]:
class_polarity = class_tanpa_polarity.copy()
n = class_polarity.shape[0]
polarity_d1 = []
polarity_d2 = []
polarity_d3 = []
for i in range(n) :
  polarity_d1.append(unvr.iloc[i+2,3])

for i in range(n) :
  polarity_d2.append(unvr.iloc[i+1,3])

for i in range(n) :
  polarity_d3.append(unvr.iloc[i,3])
class_polarity["polarity_d1"] = polarity_d1
class_polarity["polarity_d2"] = polarity_d2
class_polarity["polarity_d3"] = polarity_d3
class_polarity

In [ ]:
# !pip install xlsxwriter
# with pd.ExcelWriter("data_klasifikasi.xlsx", engine='xlsxwriter') as writer:
#     class_tanpa_polarity.to_excel(writer, sheet_name='class_tanpa_polarity', index=False)
#     class_polarity.to_excel(writer, sheet_name='class_polarity', index=False)

# **Regresi Logistik**

## Tanpa Polarity

In [ ]:
#import libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
#Load dataset
data = class_tanpa_polarity

#Membuat Variabel X dan Y
X = data[data.columns[1:4]]
Y = data['Label']

In [ ]:
# Import logistic regression classifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }

grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X, Y)

# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
y_pred = grid.predict(X)

# from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
cm_LR=confusion_matrix(Y, y_pred)
print(cm_LR)
# from sklearn.metrics import classification_report
print(classification_report(Y, y_pred))

import seaborn as sns
sns.heatmap(cm_LR, annot=True,fmt="",cmap='RdYlGn',linewidths=0.30)

## LR dengan Polarity

In [ ]:
#Load dataset
data = class_polarity

#Membuat Variabel X dan Y
X_p = data[data.columns[1:]]
Y_p = data['Label']

In [ ]:
# Import logistic regression classifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }

grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_p, Y_p)

# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
y_pred_p = grid.predict(X_p)

# from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
cm_LR=confusion_matrix(Y_p, y_pred_p)
print(cm_LR)
# from sklearn.metrics import classification_report
print(classification_report(Y_p, y_pred_p))

import seaborn as sns
sns.heatmap(cm_LR, annot=True,fmt="",cmap='RdYlGn',linewidths=0.30)

# Linear Discriminant Analysis (LDA)

In [ ]:
!pip install pingouin
import pingouin as pg
import pandas as pd

In [ ]:
data = class_tanpa_polarity
pg.box_m(data, dvs=list(data.columns)[1:], group='Label')

## Tanpa Polarity

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix

data = class_tanpa_polarity

#Membuat Variabel X dan Y
X_LDA = data[data.columns[1:4]]
Y_LDA = data['Label']

lda = LinearDiscriminantAnalysis()
model = lda.fit(X_LDA,Y_LDA)
pred = model.predict(X_LDA)

#print(confusion_matrix(Y_LDA, pred))
print(classification_report(Y_LDA, pred))

In [ ]:
import seaborn as sns
cm_LDA=confusion_matrix(Y_LDA, pred)
sns.heatmap(cm_LDA, annot=True,fmt="",cmap='RdYlGn',linewidths=0.30)

## LDA dengan Polarity

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

data = class_polarity

#Membuat Variabel X dan Y
X_LDA_p = data[data.columns[1:]]
Y_LDA_p = data['Label']

lda =  LinearDiscriminantAnalysis()
model = lda.fit(X_LDA_p,Y_LDA_p)
pred_p = model.predict(X_LDA_p)

print(confusion_matrix(Y_LDA_p, pred_p))
print(classification_report(Y_LDA_p, pred_p))

In [ ]:
import seaborn as sns
cm_LDA=confusion_matrix(Y_LDA_p, pred_p)
sns.heatmap(cm_LDA, annot=True,fmt="",cmap='RdYlGn',linewidths=0.30)

# K-Nearest Neigbour

## Tanpa Polarity

In [ ]:
#import package
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
data = class_tanpa_polarity

#menentukan variabel independen
x = data.drop(['Label'],axis = 1)

#menentukan variabel dependen
y = data['Label']

n_terbaik = 0
maks_akurasi = 0

#menentukan n terbaik KNN

for i in range(2,10):
  knn = KNeighborsClassifier(n_neighbors = i)
  knn.fit(x,y)

  y_pred = knn.predict(x)

  akurasi = accuracy_score(y, y_pred)

  print(f'Akurasi for n={i}: {akurasi}')
  if akurasi > maks_akurasi:
        maks_akurasi = akurasi
        n_terbaik = i

# Cetak nilai n_terbaik dan maks_akurasi setelah loop selesai
print(f'Nilai n terbaik: {n_terbaik}, Maksimum Akurasi: {maks_akurasi:.4f}')

In [ ]:
#KNN untuk n terbaik
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x,y)

y_pred = knn.predict(x)

cm = confusion_matrix(y, y_pred)

print('confussion matrix:')
print(confusion_matrix(y, y_pred))

print('classification report:')
print(classification_report(y, y_pred))

print('nilai akurasi:')
print(accuracy_score(y, y_pred))

In [ ]:
import seaborn as sns
sns.heatmap(cm, annot=True,fmt="",cmap='RdYlGn',linewidths=0.30)

## KNN dengan Polarity

In [ ]:
data = class_polarity

#menentukan variabel independen
x = data.drop(['Label'],axis = 1)

#menentukan variabel dependen
y = data['Label']

n_terbaik = 0
maks_akurasi = 0

#menentukan n terbaik KNN

for i in range(2,10):
  knn = KNeighborsClassifier(n_neighbors = i)
  knn.fit(x,y)

  y_pred = knn.predict(x)

  akurasi = accuracy_score(y, y_pred)

  print(f'Akurasi for n={i}: {akurasi}')
  if akurasi > maks_akurasi:
        maks_akurasi = akurasi
        n_terbaik = i

# Cetak nilai n_terbaik dan maks_akurasi setelah loop selesai
print(f'Nilai n terbaik: {n_terbaik}, Maksimum Akurasi: {maks_akurasi:.4f}')

In [ ]:
#KNN untuk n terbaik
knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(x,y)

y_pred = knn.predict(x)

cm = confusion_matrix(y, y_pred)

print('confussion matrix:')
print(confusion_matrix(y, y_pred))

print('classification report:')
print(classification_report(y, y_pred))

print('nilai akurasi:')
print(accuracy_score(y, y_pred))

In [ ]:
import seaborn as sns
sns.heatmap(cm, annot=True,fmt="",cmap='RdYlGn',linewidths=0.30)

# SVM

##SVM (Tanpa Polaritas)

In [ ]:
import pandas as pd
import numpy as np

data = class_tanpa_polarity

#Membuat Variabel X dan Y
X_SVM = data[data.columns[1:4]]
Y_SVM = data['Label']

In [ ]:
# Import SVM classifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_SVM, Y_SVM)

# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
y_pred_SVM = grid.predict(X_SVM)

# from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
cm_SVM=confusion_matrix(Y_SVM, y_pred_SVM)
print(cm_SVM)
# from sklearn.metrics import classification_report
print(classification_report(Y_SVM, y_pred_SVM))

import seaborn as sns
sns.heatmap(cm_SVM, annot=True,fmt="",cmap='RdYlGn',linewidths=0.30)

##SVM (Dengan Polarity)

In [ ]:
import pandas as pd
import numpy as np

datap = class_polarity

#Membuat Variabel X dan Y
X_SVM_p = datap[datap.columns[1:7]]
Y_SVM_p = datap['Label']

In [ ]:
# Import SVM classifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_SVM_p, Y_SVM_p)

# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
y_pred_SVM_p = grid.predict(X_SVM_p)

# from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, confusion_matrix
cm_SVM_p=confusion_matrix(Y_SVM_p, y_pred_SVM_p)
print(cm_SVM_p)
# from sklearn.metrics import classification_report
print(classification_report(Y_SVM_p, y_pred_SVM_p))

import seaborn as sns
sns.heatmap(cm_SVM_p, annot=True,fmt="",cmap='RdYlGn',linewidths=0.30)

Dapat dilihat bahwa dari keempat model klasifikasi, **Polarity akan menaikkan akurasi model jika dibandingkan dengan data tanpa polaritas. Artinya analisis sentimen dapat membantu untuk menaikkan akurasi model biasa untuk mengklasifikasian Naik/Turunnya harga saham**.